In [1]:
import pandas as pd
import numpy as np
import optuna
import os
import sys
import torch
from sklearn.model_selection import train_test_split

sys.path.insert(0, os.path.join(os.getcwd(), ".."))

from src.utils import set_random_seed
from src.dataset import Digits
from src.train import run_training, evaluate
from src.model import NeuralNetwork

%matplotlib inline

In [2]:
N_TRIALS = 50
STORAGE = "sqlite:////Users/parabam1/Projects/kaggle/digit_recognizer/study/hyperparameter_studies.db"


In [3]:
def fetch_dataset(random_seed: int, batch_size: int):

    set_random_seed(random_seed)

    dev_dataset = Digits(split="train")

    train_idx, valid_idx = train_test_split(
        np.arange(len(dev_dataset.labels)),
        test_size=0.2,
        shuffle=True,
        stratify=dev_dataset.labels,
        random_state=random_seed,
    )

    train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
    valid_sampler = torch.utils.data.SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
        dev_dataset, batch_size=batch_size, sampler=train_sampler
    )
    valid_loader = torch.utils.data.DataLoader(
        dev_dataset, batch_size=batch_size, sampler=valid_sampler
    )

    return (train_loader, valid_loader)


In [4]:
def objective(trial):

    batch_size = trial.suggest_int("batch_size", low=32, high=64)
    epochs = trial.suggest_int("epochs", low=3, high=50)
    random_seed = trial.suggest_int("random_seed", low=0, high=1000_000)
    lr = trial.suggest_loguniform("lr", low=1e-6, high=1e-4)

    train_dataloader, valid_dataloader = fetch_dataset(
        random_seed=random_seed, batch_size=batch_size
    )

    print(
        f"Epoch: {epochs}, Batch_size: {batch_size}, Learning_rate: {lr}, Random_Seed: {random_seed}"
    )

    validation_accuracy = run_training(
        train_dataloader=train_dataloader,
        valid_dataloader=valid_dataloader,
        epochs=epochs,
        lr=lr,
        random_seed=random_seed,
    )

    return validation_accuracy


In [ ]:
study = optuna.create_study(
    study_name="titanic",
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(multivariate=True),
    storage=STORAGE,
    load_if_exists=True,
)

study.optimize(
    objective,
    n_trials=N_TRIALS,
    gc_after_trial=True,
)


In [5]:
study = optuna.load_study(
    study_name="titanic",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(multivariate=True),
    storage=STORAGE,
)

train_dataloader, valid_dataloader = fetch_dataset(
    random_seed=study.best_params["random_seed"],
    batch_size=study.best_params["batch_size"],
)

run_training(
    train_dataloader=train_dataloader,
    valid_dataloader=valid_dataloader,
    epochs=39,
    lr=study.best_params["lr"],
    random_seed=study.best_params["random_seed"],
)

model = NeuralNetwork()
model.load_state_dict(
    torch.load("/Users/parabam1/Projects/kaggle/digit_recognizer/model/cnn.pth")
)


test_dataset = Digits(split="test")
accuracy, predictions = evaluate(
    model=model,
    dataloader=torch.utils.data.DataLoader(
        test_dataset, batch_size=study.best_params["batch_size"], shuffle=False
    ),
)

submission = pd.DataFrame.from_records(
    {
        "ImageID": range(1, predictions.shape[0] + 1),
        "Label": predictions.numpy().tolist(),
    }
)
submission.to_csv("../data/submission.csv", index=False)


/Users/parabam1/miniconda3/envs/kaggle/lib/python3.9/site-packages/optuna/samplers/_tpe/sampler.py:263: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(


Epoch [1/39], Loss: 0.3728, Accuracy: 94.51
Epoch [2/39], Loss: 0.1566, Accuracy: 96.70
Epoch [3/39], Loss: 0.1551, Accuracy: 97.30
Epoch [4/39], Loss: 0.1274, Accuracy: 97.79
Epoch [5/39], Loss: 0.0935, Accuracy: 98.07
Epoch [6/39], Loss: 0.0553, Accuracy: 98.31
Epoch [7/39], Loss: 0.0129, Accuracy: 98.26
Epoch [8/39], Loss: 0.1244, Accuracy: 98.39
Epoch [9/39], Loss: 0.0270, Accuracy: 98.51
Epoch [10/39], Loss: 0.0767, Accuracy: 98.62
Epoch [11/39], Loss: 0.0216, Accuracy: 98.57
Epoch [12/39], Loss: 0.0288, Accuracy: 98.67
Epoch [13/39], Loss: 0.0261, Accuracy: 98.80
Epoch [14/39], Loss: 0.0238, Accuracy: 98.79
Epoch [15/39], Loss: 0.0685, Accuracy: 98.85
Epoch [16/39], Loss: 0.0028, Accuracy: 98.83
Epoch [17/39], Loss: 0.0109, Accuracy: 98.92
Epoch [18/39], Loss: 0.0036, Accuracy: 98.85
Epoch [19/39], Loss: 0.0168, Accuracy: 98.95
Epoch [20/39], Loss: 0.0724, Accuracy: 98.96
Epoch [21/39], Loss: 0.0116, Accuracy: 98.98
Epoch [22/39], Loss: 0.0147, Accuracy: 98.93
Epoch [23/39], Loss